In [1]:
# %%
# Import necessary libraries for data handling, model utilization, and visualization
import numpy as np
import pandas as pd
import yfinance as yf  # For collecting financial data
import matplotlib.pyplot as plt
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scipy.optimize import minimize
from collections import deque
# Import the custom Model class
from Model import Model
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')

# Set the random seed for reproducibility across numpy and tensorflow
np.random.seed(123)
tf.random.set_seed(123)

plt.style.use('seaborn-darkgrid')

# Define the tickers and date range with consideration of trading days
TICKERS = ['AGG','DBC','VTI','^VIX']

# Approximate number of trading days per year (useful for annualizing returns)
TRADING_DAYS_PER_YEAR = 252
VOLATILITY_SCALING = False
# Define transaction cost rate
C = 0.0001  # 0.01%

# Confirm setup
print("Setup complete: libraries imported, random seed set, and tickers defined.")

Setup complete: libraries imported, random seed set, and tickers defined.


C:\Users\wiztu\AppData\Local\Temp\ipykernel_17820\1470264350.py:27: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [2]:
# %%
# Data Collection Step
# Objective: Fetch historical adjusted close prices for defined tickers and date range

# Download data using yfinance for the specified tickers and date range
def get_data(tickers, start_date, end_date):
    """
    Retrieves historical adjusted close prices for the given tickers and date range.
    
    Parameters:
    - tickers: List of stock ticker symbols
    - start_date: Start date for historical data
    - end_date: End date for historical data
    
    Returns:
    - DataFrame of adjusted close prices, with each column representing a ticker
    """
    # Fetch data from yfinance
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    
    # Drop rows with missing values, if any, to ensure data continuity
    data.dropna(inplace=True)
    
    return data

# Fetch the data and display a quick preview
data = get_data(TICKERS, '2006-01-01', '2020-04-30')
print("Data fetched successfully. Sample data:")
print(data.head())

# Confirm data spans the expected range and has the expected number of columns
print(f"Data covers {len(data)} trading days with {len(data.columns)} assets.")

[*********************100%%**********************]  4 of 4 completed

Data fetched successfully. Sample data:
Ticker            AGG        DBC        VTI   ^VIX
Date                                              
2006-02-06  56.300117  20.889498  44.654259  13.04
2006-02-07  56.260818  20.285255  44.219498  13.59
2006-02-08  56.232689  20.198935  44.537624  12.83
2006-02-09  56.266430  20.388842  44.452805  13.12
2006-02-10  56.148499  20.017660  44.544697  12.87
Data covers 3582 trading days with 4 assets.


In [3]:
# %%
def preprocess_data(data, rolling_window=50):
    """
    Prepares data by calculating 50-day rolling averages and returns.

    Parameters:
    - data: DataFrame of historical adjusted close prices for assets
    - rolling_window: Window size for the rolling average

    Returns:
    - normalized_data: Smoothed prices, normalized to start at 1 for each asset
    - returns: Smoothed returns using a rolling mean of percentage changes
    """
    # Calculate rolling mean for prices and returns to smooth the data
    smoothed_prices = (data.rolling(window=rolling_window).mean()).dropna()
    smoothed_returns = (data.pct_change().rolling(window=rolling_window).mean()).dropna()
    # Normalize prices to start each asset's time series at 1
    normalized_data = smoothed_prices / smoothed_prices.iloc[0]
    
    return normalized_data, smoothed_returns


# Run preprocessing and display sample data
normalized_data, smoothed_returns = preprocess_data(data)
print("Data preprocessing complete. Sample normalized data:")
print(normalized_data.head())
print("\nSample daily returns:")
print(smoothed_returns.head())


Data preprocessing complete. Sample normalized data:
Ticker           AGG       DBC       VTI      ^VIX
Date                                              
2006-04-18  1.000000  1.000000  1.000000  1.000000
2006-04-19  0.999786  1.001443  1.000798  0.997130
2006-04-20  0.999555  1.003226  1.001815  0.993875
2006-04-21  0.999332  1.005552  1.002683  0.991806
2006-04-24  0.999172  1.007249  1.003528  0.989520

Sample daily returns:
Ticker           AGG       DBC       VTI      ^VIX
Date                                              
2006-04-19 -0.000212  0.001441  0.000813 -0.001952
2006-04-20 -0.000229  0.001788  0.001035 -0.002230
2006-04-21 -0.000221  0.002295  0.000882 -0.001198
2006-04-24 -0.000157  0.001709  0.000862 -0.001374
2006-04-25 -0.000182  0.002136  0.000757 -0.000993


In [4]:
# %%
class Portfolio:
    def __init__(self, initial_cash: float, assets: pd.DataFrame):
        """
        Initializes the Portfolio object.

        Parameters:
        - initial_cash: The starting value of the portfolio in cash.
        - assets: DataFrame of asset prices (historical data).
        """
        self.initial_cash = initial_cash
        self.current_value = initial_cash
        self.assets = assets  # Historical price data for the assets
        self.weights = np.zeros(len(assets.columns))  # Initialize weights as zero
        self.portfolio_history = []  # To track portfolio value over time
        self.rebalancing_dates = []  # To store rebalancing dates

    def rebalance(self, new_weights: np.array, target_volatility = None):
        """
        Rebalances the portfolio according to new weights.

        Parameters:
        - new_weights: Numpy array of asset allocations.
        """
        if len(new_weights) != len(self.assets.columns):
            raise ValueError("Number of weights must match the number of assets.")
        self.weights = new_weights
        if target_volatility is not None:
            self.apply_volatility_scaling(target_volatility=target_volatility,rolling_window = 50)

    def calculate_initial_shares(self, initial_cash, initial_prices):
        """
        Calculates the number of shares for each asset at the start of the testing period based on
        initial cash and allocation weights.

        Parameters:
        - initial_cash: The starting cash value of the portfolio.

        Returns:
        - shares: Dictionary with tickers as keys and the initial number of shares as values.
        """
        # Calculate the dollar amount allocated to each asset
        dollar_allocation = initial_cash * self.weights

        # Calculate the number of shares for each asset
        shares = (dollar_allocation // initial_prices).astype(int)  # Floor division to get whole shares

        # Return as a dictionary for easy readability
        return dict(zip(self.assets.columns, shares))
    
    def calculate_daily_returns(self):
        """
        Applies the current weights to asset returns and updates portfolio value over time.
        """
        # Calculate daily returns for each asset
        daily_returns = self.assets.pct_change().dropna()
        
        # Calculate portfolio returns by applying weights
        portfolio_returns = np.dot(daily_returns, self.weights)

        # Track the portfolio's value over time by compounding the returns
        for daily_ret in portfolio_returns:
            self.current_value *= (1 + daily_ret)
            self.portfolio_history.append(self.current_value)

    def update_portfolio_value(self, date):
        """
        Updates the portfolio value for a single date.
        """
        # Get the index of the date
        date_index = self.assets.index.get_loc(date)
        if date_index == 0:
            # First day, no previous day to compute return
            self.portfolio_history.append(self.current_value)
            return
        # Get the asset returns for that day
        previous_date = self.assets.index[date_index - 1]
        daily_return = self.assets.loc[date] / self.assets.loc[previous_date] - 1
        # Calculate portfolio return
        portfolio_return = np.dot(daily_return.values, self.weights)
        # Update portfolio value
        self.current_value *= (1 + portfolio_return)
        # Append to history
        self.portfolio_history.append(self.current_value)


    def track_portfolio_performance(self):
        """
        Tracks and prints the portfolio performance over time.
        """
        for date, value in zip(self.assets.index[1:], self.portfolio_history):
            print(f"Date: {date}, Portfolio Value: {value}")
    
    def reset(self):
        self.current_cash = self.initial_cash
        self.assets = self.initial_assets.copy()
        self.weights = np.zeros(len(self.assets.columns))  # Reset to no investments
        self.portfolio_history = []
        self.rebalancing_dates = []
        return self.assets.iloc[0].values

    def get_portfolio_value(self):
        """
        Returns the current value of the portfolio.
        """
        return self.current_value
    
    def plot_portfolio_value(self):
        """
        Plots the portfolio value over time.
        """
        plt.figure(figsize=(10, 6))
        plt.plot(self.assets.index[1:], self.portfolio_history, label="Portfolio Value")
        plt.title("Portfolio Value Over Time")
        plt.xlabel("Date")
        plt.ylabel("Portfolio Value")
        plt.legend()
        plt.show()


In [5]:
# %%
def calculate_metrics(portfolio_values):
    """
    Calculates performance metrics for the portfolio.

    Parameters:
    - portfolio_values: List of daily portfolio values over the testing period.

    Returns:
    - metrics: Dictionary containing various performance metrics.
    """
    # Convert portfolio values to daily returns
    portfolio_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    
    # Number of days
    N = len(portfolio_returns)

    # Calculate Sharpe Ratio
    mean_return = np.mean(portfolio_returns)
    std_dev = np.std(portfolio_returns)
    sharpe_ratio = mean_return / std_dev * np.sqrt(TRADING_DAYS_PER_YEAR)
    
    # Calculate Sortino Ratio
    downside_returns = portfolio_returns[portfolio_returns < 0]
    downside_std_dev = np.std(downside_returns) if len(downside_returns) > 0 else 0
    sortino_ratio = mean_return / downside_std_dev * np.sqrt(TRADING_DAYS_PER_YEAR) if downside_std_dev != 0 else np.nan
    
    # Calculate Maximum Drawdown
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = np.max(drawdowns)
    
    # Expected return (annualized)
    cumulative_return = (portfolio_values[-1] / portfolio_values[0]) - 1
    annualized_return = (1 + cumulative_return) ** (TRADING_DAYS_PER_YEAR / N) - 1

    # Standard deviation of returns (annualized)
    annualized_std = std_dev * np.sqrt(TRADING_DAYS_PER_YEAR)

    # Percentage of positive returns
    positive_returns = portfolio_returns[portfolio_returns > 0]
    percentage_positive = len(positive_returns) / len(portfolio_returns) * 100

    # Average profit / average loss (profit/loss ratio)
    average_profit = np.mean(portfolio_returns[portfolio_returns > 0]) if len(positive_returns) > 0 else 0
    average_loss = np.mean(portfolio_returns[portfolio_returns < 0]) if len(portfolio_returns[portfolio_returns < 0]) > 0 else 0
    profit_loss_ratio = (average_profit / -average_loss) if average_loss != 0 else np.nan

    metrics = {
        "Annualized Return": annualized_return,
        "Annualized Std Dev": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Max Drawdown": max_drawdown,
        "% Positive Returns": percentage_positive,
        "Profit/Loss Ratio": profit_loss_ratio
    }
    
    return metrics


In [6]:
def volatility_scaling(weights, returns_past, target_vol=0.1):
    """
    Scales the portfolio weights based on the target volatility and ex-ante volatility estimates.

    Parameters:
    - weights: Numpy array of current portfolio weights
    - returns_past: Pandas DataFrame of past returns (50 days)
    - target_vol: Target portfolio volatility (default 10%)

    Returns:
    - scaled_weights: Numpy array of scaled portfolio weights
    """
    # Calculate exponentially weighted moving standard deviation (50-day window)
    vol_estimates = returns_past.ewm(span=50).std().iloc[-1].values  # Shape: (num_assets,)

    # Avoid division by zero
    vol_estimates = np.where(vol_estimates == 0, 1e-6, vol_estimates)

    # Calculate scaling factors: target_vol / vol_estimates
    scaling_factors = target_vol / vol_estimates

    # Apply scaling to weights
    scaled_weights = weights * scaling_factors

    # Normalize weights to sum to 1
    scaled_weights /= np.sum(scaled_weights)

    return scaled_weights


In [ ]:
# %%
def equal_weighted_strategy(returns):
    """
    Creates an equal-weighted portfolio.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - equal_weights: Numpy array of equal weights for each asset.
    """
    num_assets = returns.shape[1]
    equal_weights = np.ones(num_assets) / num_assets
    return equal_weights
# Define function to get MV weights
def mean_variance_optimized_strategy(returns):
    """
    Creates a mean-variance optimized portfolio by maximizing the Sharpe Ratio.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - optimized_weights: Numpy array of optimized weights for each asset.
    """
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    
    def neg_sharpe(weights):
        portfolio_return = np.dot(weights, mean_returns)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -portfolio_return / portfolio_std

    # Constraints: Weights must sum to 1, and each weight must be between 0 and 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(returns.shape[1]))

    result = minimize(neg_sharpe, np.ones(returns.shape[1]) / returns.shape[1], bounds=bounds, constraints=constraints)
    optimized_weights = result.x
    
    return optimized_weights

# Define function to get MD weights
def maximum_diversification(returns):
    """
    Perform maximum diversification optimization based on the given returns.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - optimal_weights: Array of portfolio weights that maximize diversification.
    """
    # Calculate asset volatilities (standard deviation of each asset’s returns)
    asset_volatilities = returns.std()

    # Calculate the covariance matrix of returns
    cov_matrix = returns.cov()

    # Define the diversification ratio to be maximized
    def neg_diversification_ratio(weights):
        # Calculate the weighted average asset volatility
        weighted_volatility = np.dot(weights, asset_volatilities)
        
        # Calculate the portfolio volatility as the weighted covariance matrix
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        
        # Diversification ratio (we negate this because we want to maximize it)
        diversification_ratio = weighted_volatility / portfolio_volatility
        return -diversification_ratio  # Negate to turn this into a minimization problem

    # Constraints: weights sum to 1, and each weight between 0 and 1 (long-only portfolio)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(len(asset_volatilities)))

    # Initial guess (equal allocation)
    init_guess = np.ones(len(asset_volatilities)) / len(asset_volatilities)

    # Optimize to find weights that maximize diversification ratio
    result = minimize(neg_diversification_ratio, init_guess, bounds=bounds, constraints=constraints)
    optimal_weights = result.x
    
    return optimal_weights


In [8]:
def average_metrics(metrics_list):
    """
    Calculates the average of each metric in the list of metrics.
    """
    avg_metrics = {}
    keys = metrics_list[0].keys()
    for key in keys:
        avg_metrics[key] = np.mean([m[key] for m in metrics_list])
    return avg_metrics


In [9]:
# %%
# Define testing periods
training_end_dates = ['2010-12-31', '2012-12-31', '2014-12-31', '2016-12-31', '2018-12-31']
testing_start_dates = ['2011-01-01', '2013-01-01', '2015-01-01', '2017-01-01', '2019-01-01']
testing_end_dates = ['2012-12-31', '2014-12-31', '2016-12-31', '2018-12-31', '2020-12-31']

periods = list(zip(training_end_dates, testing_start_dates, testing_end_dates))

# Initialize lists to store performance metrics for each model
lstm_metrics = []
mvo_metrics = []
md_metrics = []
TARGET_VOL = 0.1
initial_cash = 100000

In [ ]:
for period in periods:
    training_end, testing_start, testing_end = period
    print(f"\nProcessing period: Training up to {training_end}, Testing from {testing_start} to {testing_end}")

    # Get training data
    training_data = data.loc[:training_end].copy()
    # Get testing data
    testing_data = data.loc[testing_start:testing_end].copy()
    testing_returns = testing_data.pct_change().fillna(0)
    # Ensure we have enough data
    if len(testing_data) < 50:
        print("Not enough data for testing period")
        continue

    # LSTM Model
    # lstm_model = Model()
    # # Train the model on the training data
    # lstm_model.train(training_data)

    # # Initialize portfolio values
    # portfolio_values_lstm = [initial_cash]
    # prev_weights_lstm = None

    # # Loop through each day in the testing period for LSTM
    # for i in range(50, len(testing_data)):
    #     # Get past 50 days of data for input
    #     input_sequence = testing_data.iloc[i - 50:i]
    #     # Prepare the input (same preprocessing as during training)
    #     returns_sequence = input_sequence.pct_change().fillna(0)
    #     combined_sequence = pd.concat([input_sequence, returns_sequence], axis=1).values

    #     # Predict allocation for the day
    #     allocation = lstm_model.predict_allocation(combined_sequence)
    #     print("LSTM Allocation:", allocation)
    #     # Apply volatility scaling if needed
    #     if VOLATILITY_SCALING:
    #         # Get past returns for volatility estimation
    #         returns_past = testing_returns.iloc[i - 50:i]
    #         allocation = volatility_scaling(allocation, returns_past, target_vol=TARGET_VOL)

    #     # Get today's returns from precomputed returns
    #     today_return = testing_returns.iloc[i].values

    #     # Compute transaction costs
    #     if prev_weights_lstm is None:
    #         transaction_cost = C * np.sum(np.abs(allocation))
    #     else:
    #         transaction_cost = C * np.sum(np.abs(allocation - prev_weights_lstm))

    #     # Update portfolio value
    #     portfolio_return = np.dot(allocation, today_return)
    #     new_value = portfolio_values_lstm[-1] * (1 + portfolio_return) - transaction_cost * portfolio_values_lstm[-1]
    #     portfolio_values_lstm.append(new_value)

    #     # Update previous weights
    #     prev_weights_lstm = allocation

    # # Calculate and store performance metrics for LSTM
    # metrics_lstm = calculate_metrics(portfolio_values_lstm)
    # lstm_metrics.append(metrics_lstm)

    # Mean-Variance Optimization (MVO) Model
    # Precompute rolling metrics (done outside loop for efficiency if possible)
    # Here it's recomputed per period, which may not be necessary
    returns_full_testing = data.loc[:testing_end].pct_change().dropna()
    returns_testing = data.loc[testing_start:testing_end].pct_change().dropna()

    # Get indices of testing dates in returns
    testing_indices = returns_full_testing.index.get_indexer_for(returns_testing.index)

    portfolio_values_mv = [initial_cash]
    prev_weights_mv = None

    for i in range(50, len(testing_indices)):  # Start after the first 50 days of testing
        # Get the past 50 days of rolling means
        input_data = returns_full_testing.iloc[i - 50:i]
        weights_mv = mean_variance_optimized_strategy(input_data)

        # Apply volatility scaling if required
        if VOLATILITY_SCALING:
            weights_mv_scaled = volatility_scaling(weights_mv, returns_full_testing.iloc[i - 50:i], target_vol=TARGET_VOL)
        else:
            weights_mv_scaled = weights_mv

        print("MVO Allocation:", weights_mv_scaled)
        # Today's returns
        today_return = returns_testing.iloc[i].values

        # Compute transaction costs
        if prev_weights_mv is None:
            transaction_cost = C * np.sum(np.abs(weights_mv_scaled))
        else:
            transaction_cost = C * np.sum(np.abs(weights_mv_scaled - prev_weights_mv))

        # Update portfolio value
        portfolio_return = np.dot(weights_mv_scaled, today_return)
        portfolio_value = portfolio_values_mv[-1] * (1 + portfolio_return) - transaction_cost * portfolio_values_mv[-1]
        portfolio_values_mv.append(portfolio_value)

        # Update previous weights
        prev_weights_mv = weights_mv_scaled

    # Calculate and store performance metrics for MVO
    metrics_mv = calculate_metrics(portfolio_values_mv)
    mvo_metrics.append(metrics_mv)

    # Maximum Diversification (MD) Model
    portfolio_values_md = [initial_cash]
    prev_weights_md = None

    for i in range(50, len(testing_indices)):  # Start after the first 50 days of testing
        input_data = returns_full_testing.iloc[i - 50:i]
        weights_md = maximum_diversification(input_data)
        # Apply volatility scaling if required
        if VOLATILITY_SCALING:
            weights_md_scaled = volatility_scaling(weights_md, returns_full_testing.iloc[i - 50:i], target_vol=TARGET_VOL)
        else:
            weights_md_scaled = weights_md

        print("MD Allocation:", weights_md_scaled)
        # Today's returns
        today_return = returns_testing.iloc[i].values

        # Compute transaction costs
        if prev_weights_md is None:
            transaction_cost = C * np.sum(np.abs(weights_md_scaled))
        else:
            transaction_cost = C * np.sum(np.abs(weights_md_scaled - prev_weights_md))

        # Update portfolio value
        portfolio_return = np.dot(weights_md_scaled, today_return)
        portfolio_value = portfolio_values_md[-1] * (1 + portfolio_return) - transaction_cost * portfolio_values_md[-1]
        portfolio_values_md.append(portfolio_value)

        # Update previous weights
        prev_weights_md = weights_md_scaled

    # Calculate and store performance metrics for MD
    metrics_md = calculate_metrics(portfolio_values_md)
    md_metrics.append(metrics_md)


Processing period: Training up to 2010-12-31, Testing from 2011-01-01 to 2012-12-31


ValueError: shapes (4,4) and (50,) not aligned: 4 (dim 1) != 50 (dim 0)

In [ ]:
# Calculate average metrics for each model
# lstm_avg_metrics = average_metrics(lstm_metrics)
mvo_avg_metrics = average_metrics(mvo_metrics)
md_avg_metrics = average_metrics(md_metrics)

In [ ]:
# # Print the average metrics
# print("\nAverage Metrics for LSTM Model:")
# for key, value in lstm_avg_metrics.items():
#     print(f"{key}: {value:.4f}")

print("\nAverage Metrics for MVO Strategy:")
for key, value in mvo_avg_metrics.items():
    print(f"{key}: {value:.4f}")

print("\nAverage Metrics for MD Strategy:")
for key, value in md_avg_metrics.items():
    print(f"{key}: {value:.4f}")


Average Metrics for MVO Strategy:
Annualized Return: 0.0930
Annualized Std Dev: 0.0868
Sharpe Ratio: 1.2788
Sortino Ratio: 1.9150
Max Drawdown: 0.0826
% Positive Returns: 54.9369
Profit/Loss Ratio: 1.0598

Average Metrics for MD Strategy:
Annualized Return: 0.1392
Annualized Std Dev: 0.0902
Sharpe Ratio: 1.8799
Sortino Ratio: 3.0918
Max Drawdown: 0.0777
% Positive Returns: 55.6681
Profit/Loss Ratio: 1.1472
